In [2]:
#on charge le dataset et on le met dans un dataframe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('Data.csv', sep=',', low_memory=False)



In [3]:
# On ne peut pas garder toutes les colonnes, un choix s'opère donc. 

# Nous scindons les colonnes restantes dans plusieurs dataframes :

# Data, dataframe général, possédant "eventid","iyear","imonth","iday","extended","resolution","country_txt","region_txt","provstate","city","success", "suicide","attacktype1_txt", "targtype1_txt","targsubtype1_txt","natlty1_txt","gname","nperps","claimed","weaptype1_txt","weapsubtype1_txt","nkill","nwound","dbsource"
Data = df[["eventid","iyear","imonth","iday","extended","resolution","country_txt","region_txt","provstate","city","success", "suicide","attacktype1_txt", "targtype1_txt","targsubtype1_txt","natlty1_txt","gname","nperps","claimed","weaptype1_txt","weapsubtype1_txt","nkill","nwound","dbsource"]]



In [13]:
# geo, dataframe ayant pour but la géolocalisation, comprenant "eventid", "latitude", "longitude"
geo = df[["eventid", "latitude", "longitude"]]

#geoFull correspond au dataframe geo, mais avec les valeurs nulles enlevées

geoFull = df[["eventid", "latitude", "longitude"]].dropna()

geoFull.to_csv("Geo.csv")

In [5]:
#headlines, dataframe comprenant les descriptions. "eventid","summary"
headlines = df[["eventid","summary"]]

On va tester les données géographiques avec le dataframe geo

In [6]:
geo.describe()

,eventid,latitude,longitude
count,1.816910e+05,177135.000000,1.771340e+05
mean,2.002705e+11,23.498343,-4.586957e+02
std,1.325957e+09,18.569242,2.047790e+05
min,1.970000e+11,-53.154613,-8.618590e+07
25%,1.991021e+11,11.510046,4.545640e+00
50%,2.009022e+11,31.467463,4.324651e+01
75%,2.014081e+11,34.685087,6.871033e+01
max,2.017123e+11,74.633553,1.793667e+02


In [7]:
import folium
import pandas

# test des données géographiques du dataframe geo
#on veut afficher une map du monde avec un point pour chaque ligne du dataframe

#pour ce faire on va : 


In [16]:
# On regarde le nombre de d'attentats revendiqués par rapport au nombre total 

Revendiqué = df[["claimed"]]

#On récupère le nombre de ligne ou la colonne "claimed" est égale à 1 dans la variable "count"

count = Revendiqué[Revendiqué.claimed == 1].count()

#On récupère le nombre de ligne total dans la variable "count2"

count2 = Revendiqué.count()

#On calcule le pourcentage de ligne ou la colonne "claimed" est égale à 1

pourcentage = (count/count2)*100

print(pourcentage)


claimed    16.445302
dtype: float64


In [19]:
 # On regarde le nombre d'attentats par année pour voir comment évolue le terrorisme au fil des années

#On récupère le nombre d'attentats par année dans la variable "YearCount"

YearCount = df[["eventid","iyear"]].groupby('iyear').count()

# On affiche le nombre d'attentats par année

print(YearCount)

       eventid
iyear         
1970       651
1971       471
1972       568
1973       473
1974       581
1975       740
1976       923
1977      1319
1978      1526
1979      2662
1980      2662
1981      2586
1982      2544
1983      2870
1984      3495
1985      2915
1986      2860
1987      3183
1988      3721
1989      4324
1990      3887
1991      4683
1992      5071
1994      3456
1995      3081
1996      3058
1997      3197
1998       934
1999      1395
2000      1814
2001      1906
2002      1333
2003      1278
2004      1166
2005      2017
2006      2758
2007      3242
2008      4805
2009      4721
2010      4826
2011      5076
2012      8522
2013     12036
2014     16903
2015     14965
2016     13587
2017     10900
